# 二創 Scenario

## 1. Initialize agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"
API_KEY = "carol_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

IPython autoawait is `on`, and set to use `asyncio`


## 2. DM-SP did exchange 

### Build connection

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


In [3]:
##############
# Invitation #
##############

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3eca5c09-a8df-4db3-8ca8-3c42e90e938a', 'label': 'Alice', 'recipientKeys': ['AuLKwuoT5PuDn8Qo4AeBH78eRzwGyBnq1xshmEbRPgGV'], 'serviceEndpoint': 'http://192.168.65.3:8020'}

In [4]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]

Connection Handler Called
Connection 1fac8840-4a32-46b6-9c33-6da407bfc598 in State invitation
Connection Handler Called
Connection 1fac8840-4a32-46b6-9c33-6da407bfc598 in State request
Connection {'connection_id': '1fac8840-4a32-46b6-9c33-6da407bfc598', 'my_did': '6pQ7V5XsZMfTHqYq3PTGzX', 'created_at': '2023-01-05 12:00:22.405815Z', 'their_label': 'Alice', 'invitation_mode': 'once', 'rfc23_state': 'request-sent', 'state': 'request', 'invitation_key': 'AuLKwuoT5PuDn8Qo4AeBH78eRzwGyBnq1xshmEbRPgGV', 'routing_state': 'none', 'their_role': 'inviter', 'accept': 'manual', 'request_id': '550ff564-baa9-4a04-b6c3-bc8a2da10f00', 'updated_at': '2023-01-05 12:00:22.480362Z'}
Connection Handler Called
Connection 1fac8840-4a32-46b6-9c33-6da407bfc598 in State response
Connection Handler Called
Connection 1fac8840-4a32-46b6-9c33-6da407bfc598 in State active


### > Break here - switch to DM accept request <

### Send DID to DM

In [5]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])
dm_connection = connection["connection_id"]
print(dm_connection)

Alice AGENT CONNECTION
{'connection_id': '1fac8840-4a32-46b6-9c33-6da407bfc598', 'my_did': '6pQ7V5XsZMfTHqYq3PTGzX', 'created_at': '2023-01-05 12:00:22.405815Z', 'their_label': 'Alice', 'invitation_mode': 'once', 'rfc23_state': 'completed', 'state': 'active', 'their_did': 'EHZb7jPbXpTm7Jixc94kGz', 'invitation_key': 'AuLKwuoT5PuDn8Qo4AeBH78eRzwGyBnq1xshmEbRPgGV', 'routing_state': 'none', 'their_role': 'inviter', 'accept': 'manual', 'request_id': '550ff564-baa9-4a04-b6c3-bc8a2da10f00', 'updated_at': '2023-01-05 12:00:37.869352Z'}
State: active
1fac8840-4a32-46b6-9c33-6da407bfc598


In [6]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

my DID X8HWQruWXrv8s6BBLXm53B


In [7]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', dm_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


In [8]:
response = await agent_controller.messaging.send_message(dm_connection, 'sp_'+my_did)
print(response)

{}
Handle message {'connection_id': '1fac8840-4a32-46b6-9c33-6da407bfc598', 'message_id': '66f87f11-7fca-490b-a37b-7705102007b7', 'content': 'PQRXDxdGqQGSZ8z69p4xZP', 'state': 'received'} 1fac8840-4a32-46b6-9c33-6da407bfc598
DM DID PQRXDxdGqQGSZ8z69p4xZP


In [9]:
print(dm_did)
print(my_did)

PQRXDxdGqQGSZ8z69p4xZP
X8HWQruWXrv8s6BBLXm53B


## 3. Verify sp did

### Basic setup - listener and connection check

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    global presentation_exchange_id
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
    if state == "presentation_received":
     
        presentation_exchange_id = pres_ex_id 
        print(presentation_exchange_id)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)


In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("#######################")
print("Invitation - Copy this!!")
print(invite_message)

### > Break: Go to MUSIC to setup <

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

In [ ]:
#####################
# get MUSIC connection #
#####################

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

### Send proof request

In [ ]:
try:
    response = await agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connections = response['results']
        print("Connection :", connections)
        for connection in connections:
            if connection['state'] == 'active' and connection["their_label"] == "Bob":
                connection_id = connection["connection_id"]
                print("\nActive Connection ID : ", connection_id)
            else:
                print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

In [ ]:
response = await agent_controller.wallet.get_public_did()
issuer_did = dm_did
print("issuer: dm, has ", dm_did)

print("Request proof of Name and Age range from Bob")
#Set some variables

exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "origin", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "music", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "owner", "restrictions": [{"issuer_did": issuer_did}]},
]
# req_attrs = [
#     {"name": "origin", "restrictions": [{"issuer_did": issuer_did, "credential_proposal": {
#         "attributes": [{"name": "music", "value": "XiUyVvSwqkU5JfbvFpjuCY"}]
#     }}]},
#     {"name": "music", "restrictions": [{"issuer_did": issuer_did, "credential_proposal": {
#         "attributes": [{"name": "music", "value": "XiUyVvSwqkU5JfbvFpjuCY"}]
#     }}]},
#     {"name": "owner", "restrictions": [{"issuer_did": issuer_did, "credential_proposal": {
#         "attributes": [{"name": "music", "value": "XiUyVvSwqkU5JfbvFpjuCY"}]
#     }}]},
# ]


#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
# presentation_exchange_id = response['presentation_exchange_id']
print("\n")
# print(presentation_exchange_id)

### >Break: Finish sending proof, check MUSIC for the proof request!<

### Verify the result back

In [ ]:

verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

print(verify)

In [ ]:
origin_did = verify["presentation"]["requested_proof"]["revealed_attrs"]["0_origin_uuid"]["raw"]
print("Copy this to dm scenario3")
print(origin_did) # sign this to VC for recreation
music_did = verify["presentation"]["requested_proof"]["revealed_attrs"]["0_music_uuid"]["raw"]
print("Copy this to dm scenario3")
print(music_did) # sign this to VC for recreation 
owner_did = verify["presentation"]["requested_proof"]["revealed_attrs"]["0_owner_uuid"]["raw"]
print(owner_did) # Find owner by this

## 4. Receive and store VC for recreation - SP is the Holder

In [ ]:
from datetime import datetime
format = "%Y-%m-%d %H:%M:%S.%fZ"
response = await agent_controller.issuer.get_records()
results = response["results"]
results = list(filter(lambda r: r["state"] == "offer_received", results))
results = sorted(
        results,
        key=lambda c: datetime.strptime(c["created_at"], format),
        reverse=True,
    )
# print(results)
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
    
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

In [ ]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
# Store credential
response = await agent_controller.issuer.store_credential(cred_ex_id, "Recreation Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

## 5. SP wants to save its recreation-artpiece on the platform - Needs to pass the VC check! - SP is the proofer

### Issue a recreation

In [ ]:
# re-creator generate the song
response = await agent_controller.wallet.create_did()
print(response)
music_did = response["result"]["did"]
verkey = response["result"]["verkey"]

# Define you schema name - must be unique on the ledger
schema_name = "ownership_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "music", "time", "version", "author", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)

In [ ]:
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did=nym["result"]["did"])
print(created_schema)
schema_id = created_schema["schema_ids"][-1]
print(schema_id)

In [ ]:
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][-1]

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active':       
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

In [ ]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
response = await agent_controller.wallet.create_did()
print(response)
music_did = response["result"]["did"]
credential_attributes = [
    {"name": "author", "value": nym["result"]["did"]},
    {"name": "music", "value": music_did},
    {"name": "time", "value": time},
    {"name": "origin", "value": origin_did},
    {"name": "owner", "value": nym["result"]["did"]},
    {"name": "version", "value": "v1"}
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

### Verify the Recreation-VC

In [ ]:
# Song exists?
credential_id = "Recreation Credential"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

In [ ]:
from datetime import datetime
response = await agent_controller.proofs.get_records()
# print(response)

format = "%Y-%m-%d %H:%M:%S.%fZ"


response["results"] = list(filter(lambda r: r["state"] == "request_received", response["results"]))
response["results"] = sorted(
        response["results"],
        key=lambda c: datetime.strptime(c["created_at"], format),
        reverse=True,
    )
print('\n')

state = response['results'][-1]["state"]
presentation_exchange_id = response['results'][-1]['presentation_exchange_id']
presentation_request = response['results'][-1]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][-1]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][-1]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][-1]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][-1]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

In [ ]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)

credentials = list(filter(lambda credential: credential["cred_info"]["referent"] == "Recreation Credential", credentials))
print(credentials)


if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    print(referent)
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

### Send proof back!

In [ ]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

### Wait for Credential Exchange

In [ ]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

In [ ]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

# Terminate Agent

In [ ]:
response = await agent_controller.terminate()
print(response)